### Set Up Packages and Global Settings

In [1]:
import pytz
import random
import time

from datetime import datetime

import fastf1 as ff1
import pandas as pd

pd.set_option("display.max_columns", None)

### Explore Schedule

In [ ]:
schedule = ff1.get_event_schedule(1994)
schedule.RoundNumber.to_list()

### Explore Session Object

In [ ]:
session_quali = ff1.get_session(2020, 1, "Q")

session_quali.load()
session_quali.event

In [ ]:
session_quali = ff1.get_session(2020, 1, 'R')

session_quali.load()
session_quali.results

### Explore Laps

In [ ]:
lap_data = session_number.laps
print(lap_data)

In [ ]:
lap_data.pick_team('McLaren').pick_fastest()

In [ ]:
lap_data.pick_drivers(['PIA', 'NOR'])

### Compare DriverId field to names

In [ ]:
df_all = pd.DataFrame()

list_years = list(range(1994, 1995 + 1))

max_retries = 5

for year in list_years:
    print(f"Pulling data for {year}")

    schedule = ff1.get_event_schedule(year)
    rounds_all = schedule.RoundNumber.to_list()
    rounds_races = [round for round in rounds_all if round > 0]

    for round in rounds_races:
        for attempt in range(max_retries):
            try:
                now_utc = datetime.now(pytz.timezone("UTC"))
                now_cst = now_utc.astimezone(pytz.timezone("America/Chicago"))

                print(f"Pulling data for round {round} at {now_cst}...")

                session_quali = ff1.get_session(year, round, "Q")
                session_quali.load()
                df_quali = session_quali.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_quali, ignore_index=True)
                time.sleep(5)

                session_race = ff1.get_session(year, round, "R")
                session_race.load()
                df_race = session_race.results[["DriverId", "LastName", "FirstName"]]

                df_all = df_all.append(df_race, ignore_index=True)
                time.sleep(5)

                df_all = df_all.drop_duplicates().reset_index(drop=True)

                # If no exception was raised, break the loop
                break
            except Exception as e:
                wait_time = (2**attempt) + random.random()
                print(
                    f"Could not load data for round {round} due to {e}. Retrying in {wait_time} seconds."
                )
                time.sleep(wait_time)
                continue

df_unique = df_all.sort_values(by="DriverId")

### Develop Approach for Quali Results Object

In [16]:
session_quali = ff1.get_session(2023, 1, "Q")

session_quali.load()
df_raw = session_quali.results[
    ["Q1", "Q2", "Q3", "DriverId", "LastName", "FirstName", "TeamName", "Position"]
]

df_processed = df_raw.melt(
    id_vars=["DriverId", "LastName", "FirstName", "TeamName", "Position"],
    value_vars=["Q1", "Q2", "Q3"],
    var_name="session",
    value_name="time",
)

df_processed.columns = df_processed.columns.str.lower()

df_processed["originalposition"] = df_processed["position"]
df_processed["position"] = df_processed.groupby("session")["time"].rank(
    method="min", ascending=True
)
df_processed["position"] = df_processed["position"].fillna(
    df_processed["originalposition"]
)

df_processed = df_processed.drop(columns=["originalposition"])
df_final = df_processed[[
    "driverid",
    "lastname",
    "firstname",
    "teamname",
    "session",
    "position",
    "time",
]]

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '55', '14', '63', '44', '18', '31', '27', '4', '77', '24', '22', '23', '2', '20', '81', '21', '10']


### Develop Approach for Race Results Object

In [15]:
session_quali = ff1.get_session(2023, 1, "R")

session_quali.load()
df_raw = session_quali.results[
    [
        "DriverId",
        "LastName",
        "FirstName",
        "TeamName",
        "ClassifiedPosition",
        "Time",
    ]
]

df_processed = df_raw.copy()

df_processed["session"] = "Race"

df_processed.rename(columns={"ClassifiedPosition": "position"}, inplace=True)
df_processed.columns = df_processed.columns.str.lower()
df_final = df_processed[
    [
        "driverid",
        "lastname",
        "firstname",
        "teamname",
        "session",
        "position",
        "time",
    ]
]

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


In [37]:
session_quali.event[["RoundNumber", "Location", "EventDate", "Country"]]

RoundNumber                      1
Location                    Sakhir
EventDate      2023-03-05 00:00:00
Country                    Bahrain
dtype: object